# Metagenomic Assembly

This notebook carries out assembly of the metagenomic samples, prediction and clustering of proteins from the assemblies, and functional annotation using EggNog_Mapper. 

Sample identifiers can be resolved using the following table:

| Sample | Replicate 1 | Replicate 2 | Replicate 3 |
|--------|-------------|-------------|-------------|
| Elite  | 2000        | 2001        | 2002        |
| Desert | 2006        | 2007        | 2009        |
| North  | 2011        | 2012        | 2013        |
| Bulk   | 2023        | 2024        | 2025        |

Note that many processes in this protocol are highly resource intensive, and are run by submission of separate bash scripts to an HPC cluster. These are optimised for the University of Dundee HPC cluster running Univa Grid Engine, and will require adjustment to run in other environments.


Quality trimmed fastq files are placed in a `fastq` subdirectory, and are named `$sample_[12].fq.gz`

## 1. Setup

Python session setup...

In [115]:
import pandas as pd
import pysam
import glob
import os
from io import StringIO
from Bio import SeqIO, bgzf
from gzip import open as gzopen

samples=('2000', '2001', '2002', '2006', '2007', '2009', '2011', '2012', '2013', '2023', '2024', '2025')
sample_replicates={
    'Elite':  ['2000','2001','2002'],
    'Desert': ['2006','2007','2009'],
    'North':  ['2011','2012','2013'],
    'Bulk':   ['2023','2024','2025']
}

## 2. Host Genome Removal 

QC-filtered sequence reads were initially aligned against the barley genome (Barley_Morex_V2_pseudomolecules) using BWA-MEM, and reads mapping to the genome discarded.

### 2(a): BWA Indexing

Indexing is carried out using the 'bwa_index.sh' script as follows:

In [ ]:
qsub bin/bwa_index.sh -r reference/Barley_Morex_V2_pseudomolecules

## 2(b): BWA alignment

Alignment to the barley reference genome was carried out using the `align_reference.sh` script, as follows:

In [ ]:
!qsub -t 1-12 bin/align_reference.sh -d reference -r Barley_Morex_V2_pseudomolecules -f fastq -o barley_alignments

## 2(c): Barley Genome Filtering

Unaligned reads with extracted from the bam files using `samtools fastq`, selecting unaligned reads with the SAM flag 12 (read unmapped and mate unmapped) and stored in `fastq/non_barley_reads`. The script was submitted as:

In [ ]:
!qsub -t 1-12 bin/get_unaligned.sh -i barley_alignments -o fastq/non_barley

## 2(d): Duplicate Removal

Clumpify was run on the barley-filtered reads to remove duplicates - outputs are stored in `fastq/non_barley`. The script was submitted as:

In [ ]:
!qsub -t 1-12 bin/ref_free_dedup.sh -f fastq/non_barley/ -o fastq/deduped

## 2(e): Downsampling

Following removal of sequences mapping to the barley genome and removal of duplicate reads, the smallest resulting library size was ~13.6M reads. To produce comparable outputs, all deduplicated libraries were downsampled to this size using bbtools `reformat.sh` script, with the results written to `fastq/downsampled`. The script was submitted as:

In [ ]:
!qsub -t 1-12 bin/downsample.sh -i fastq/deduped/ -o fastq/downsampled -c 13600000

# 3: Assembly

Assembly was carried out on the downsampled reads using metaspades, with default (auto-selected) k-mer sizes. Two rounds of assembly were carried out, the first per-replicate. Following assembly, reads not integrated into the assembly were separated and made into per-sample pools by aligning reads against the assembly, extracting the reads which do not map and pooling the resulting reads from each replicate at the sample leve. These were subjected to a second round of assembly and enabling lower abundance sequences to be captured.

## 3(a): Round 1 Assembly

In [ ]:
!qsub -t 1-12 bin/assemble.sh -i fastq/downsampled/ -o assemblies/round1

### 3(b): Creating BWA indices

A set of BWA indices is required for each set of assembled contigs in order to align the reads back against the assembled contigs with BWA. The contigs are symlinked into a `bwa_index` subdirectoy of `assemblies`, and `bwa_index.sh` (see section 2(a)) run on each set of contigs.

In [38]:
%%bash

readarray -t samples < <(ls -1 assemblies/round1|cut -f1 -d_|sort -u)
assem_dir=$(readlink -f assemblies)

mkdir ${assem_dir}/bwa_index

for sample in ${samples[@]}; do
    mkdir ${assem_dir}/bwa_index/${sample}
    ln -s ${assem_dir}/round1/${sample}/contigs.fasta ${assem_dir}/bwa_index/${sample}/contigs.fasta
    qsub bin/bwa_index.sh -r ${assem_dir}/bwa_index/${sample}/contigs.fasta
done

### 3(c): Read alignment to contigs
Alignment is carried out with BWA, using the same `align_reference.sh` script as in section 2(b). Since the index path is different for each sample, a separate submission is made for each sample, with a single task per job

In [ ]:
%%bash
for sample in $(ls assemblies/round1/); do
    qsub bin/align_reference.sh -d assemblies/bwa_index/$sample -r contigs.fasta -f fastq/downsampled/ -o assemblies/alignments
done

### 3(d): Extract unaligned reads

Unaligned reads are extracted from the bam alignments suing the same `get_unaligned.sh` script as in section 2(c).

In [ ]:
!qsub -t 1-12 bin/get_unaligned.sh -i assemblies/alignments/ -o assemblies/unaligned_reads

### 3(e): Create per-sample replicate pools

Separate pools of reads for each sample were created by combining the unassembled reads from each of the samples replicates. This gives greater depth to allow rarer sequences to be assembled. 

In [73]:
os.mkdir("assemblies/unaligned_pools")
for sample in sample_replicates.keys():
    for read in ('1','2'):
        sample_records=[]
        for rep in sample_replicates[sample]:
            for record in SeqIO.parse( gzopen("assemblies/unaligned_reads/{}.filtered_{}.fq.gz".format(rep,read), "rt"), format="fastq"):
                sample_records.append(record)

        with bgzf.BgzfWriter("assemblies/unaligned_pools/{}_{}.fq.gz".format(sample,read), "wb") as outgz:
           SeqIO.write(sequences=sample_records, handle=outgz, format="fastq")

### 3(f): SPAdes Assembly

Assembly was carried out as for the initial per-sample assemblies, using `assemble.sh`.

In [ ]:
!qsub -t 1-4 bin/assemble.sh -i assemblies/unassembled_pools/ -o assemblies/round2

# 4: Protein Prediction and Clustering

Predicted proteins are generated using prodigal, with the `bin/prodigal_predict.sh` script, which simply takes an input and output directory as arguments. This will produce both nucleotide and protein fasta files for the predicted genes, and a gff file, which are copied to the output directory.

## 4(a): Round 1 Predictions

In [ ]:
!qsub -t 1-12 bin/prodigal_predict.sh -i assemblies/round1 -o protein_predictions

## 4(b): Round 2 Predictions

In [ ]:
!qsub -t 1-4 bin/prodigal_predict.sh -i assemblies/round2 -o protein_predictions

## 4(c): Assign Unique IDs

The predictions from all samples within each assembly need to be merged prior to clustering. Although with spades naming convention ID collisions are unlikely, we should first modify the sequence IDs to include the sample name to ensure uniqueness, and if necessary to later identify which assembly a prediction arose from. 

In [ ]:
%%bash

function rename_preds {
    assemblies=$1

    for assembly in ${assemblies[@]}; do
        sed -i.bak "s/>/>${assembly}_/" protein_predictions/${assembly}.nucl.fa;
        sed -i.bak "s/>/>${assembly}_/" protein_predictions/${assembly}.proteins.fa;
    done
}

asm_dir='assemblies/round1'
readarray -t assemblies < <(ls ${asm_dir})
rename_preds $assemblies

asm_dir='assemblies/round2'
readarray -t assemblies < <(ls ${asm_dir})
rename_preds $assemblies

After checking this has worked ok, remove the backup files created by sed:

In [79]:
! rm protein_predictions/*.bak

## 4(d): Pool protein predictions

Now, create a single file of protein predictions for each environment/condition, i.e all separate round1 replicates, and the round2 assembly from each condition.

In [ ]:
pred_dir='protein_predictions'
pool_dir='protein_predictions/pooled'
os.mkdir(pool_dir)

for pool in sample_replicates.keys():
    samples=sample_replicates[pool]
    samples.append(pool)
    with open("{}/{}.nucl.fa".format(pool_dir,pool), "w") as handle:
        for sample in samples:
            for record in SeqIO.parse('{}/{}.nucl.fa'.format(pred_dir,sample),'fasta'):
                SeqIO.write(record,handle,'fasta')
                
    with open("{}/{}.proteins.fa".format(pool_dir,pool), "w") as handle:
        for sample in samples:
            for record in SeqIO.parse('{}/{}.proteins.fa'.format(pred_dir,sample),'fasta'):
                SeqIO.write(record,handle,'fasta')
                
with open ("{}/all.nucl.fa".format(pool_dir),'w') as handle:
    for pool in sample_replicates.keys():
        for record in SeqIO.parse('{}/{}.nucl.fa'.format(pool_dir,pool),'fasta'):
            SeqIO.write(record,handle,'fasta')
            
with open ("{}/all.proteins.fa".format(pool_dir),'w') as handle:
    for pool in sample_replicates.keys():
        for record in SeqIO.parse('{}/{}.proteins.fa'.format(pool_dir,pool),'fasta'):
            SeqIO.write(record,handle,'fasta')
    

## 4(e): Cluster Gene Predictions 

All protein predictions are clustered using MMseqs2 with the `mmseqs_cluster.sh` script, which uses MMseqs2 easy-cluster method, which produces a fasta file of representative sequences from each cluster.

In [ ]:
!qsub bin/mmseqs_cluster.sh -i protein_predictions/pooled/all.proteins.fa -o protein_clusters

# 5: Functional annotation and abundance estimation

## 5(a): EggNog Annotations

eggnog_mapper is used to assign functional classifications to the representative sequences of each cluster, through the `eggnog_mapper.sh` script.  The eggnog_data directory contains the eggnog_proteins.dmnd and eggnog.db.gz files from version 2.0.1 of eggnog mapper with DB version 2.0.

In [ ]:
!qsub bin/eggnog_mapper.sh -i protein_clusters/mmseqs_rep_seq.fasta -o eggnog_annotations -e ../eggnog_data/

## 5(b): Read mapping to representative cDNAs

To determine per-sample abundance of the clustered protein sequences, the original reads for each replicate can be aligned against the cDNA sequences associated with the protein predictions with BWA. First the set of cDNA sequences matching the annotated representative protein sequences from each cluster needs to be obtain. Since the IDs of the cDNA and proteins match, this should be a straightforward question of pulling out the cDNA sequences with matching IDs to the proteins. The `get_rep_cdna.py` script does this.

In [ ]:
!qsub bin/get_rep_cdna.py --cdna protein_predictions/pooled/all.nucl.fa --repprot protein_clusters/mmseqs_rep_seq.fasta --repcdna protein_clusters/mmseqs_rep_cDNA_seq.fasta

## 5(c): BWA Indexing

The representative cDNA sequences are now BWA indexed using the same `bwa_index.sh` script as in section 2(a).

In [ ]:
!qsub bin/bwa_index.sh -r protein_clusters/mmseqs_rep_cDNA_seq.fasta

## 5(d): BWA Alignment

BWA alignment of the original, unrarified reads was carried out against the cDNA representative sequences using the `align_reference.sh` script as in section 2(b)

In [ ]:
!qsub -t 1-12 bin/align_reference.sh -f fastq/ -o protein_clusters/alignments -d protein_clusters -r mmseqs_rep_cDNA_seq.fasta

## 5(e): Filtering

Due to the patchy nature of metagenomic assembly and resulting short peptide sequences, there are various different classes of read alignment which can be considered valid when enumerating the aligned reads:

1)  Reads which are correctly paired and aligned to a single sequence
2)  Reads which only have one of the read-pairs aligned to a reference (singletons). In normal genomic mapping these would be discarded, however a well mapped singleton is potentially valid when aligned to a short reference
3)  Reads which are aligned to different reference sequences. These would also traditioanlly be discarded, however with the short cDNA sequences this is also potentially a valid scenario, and ~25% of the reads fall into this category. 

Each of these cases will be addressed separately, so the alignments are first filtered and split into three sets. In all cases, a mapping quality >30 is required to remove any multimapping reads which may confound analysis. The criteria for filtering are as follows:

*  Correctly paired reads: Reads are mapped as proper pair (SAM flag 2) and are primary alignments (not SAM flag 256) - saved as `${sample}.proper.bam`
*  Singletons: Mate is unmapped (SAM flag 8) and is primary alignment (not SAM flag 256) - saved as `${sample}.singletons.bam`
*  Separate reference sequence: The 'chromosome' and 'next chromosome' fields do not contain '=' - saved as `${sample}.diff_cdna.bam`

In [ ]:
!qsub -t 1-12 bin/filter_cdna_alignments.sh -d protein_clusters/alignments/

## 6: cDNA counts


Count aligned reads to each cDNA, accepting either correctly paired reads, or singletons which may well be unable to map pairs due to short target lengths. This doesn't count paired reads with reads on alternate targets, which need a bit more thinking about to only accept these if they GO terms are consistent between the two sequences.

The get_counts function below is used to extract these values from the samtools idxstats outputs generated by `align_reference.sh`.

In [142]:
def get_counts(align_dir: str, sample:str):
    
    """
    Obtains counts of properly paired and singleton reads for a sample from samtools idxstats output
    
    Required parameters:
    align_dir (str): path to directory containing idxstats outputs
    sample (str): sample name
    
    Returns:
        counts(pd.DataFrame): Pandas dataframe of counts
    """
    
    colnames=('#query_name','length','paired_count','whoknows')
    proper_pairs_counts=pd.read_csv(
        '{}/{}.proper.idxstats'.format(align_dir,sample), sep="\t",names=colnames)
    proper_pairs_counts=proper_pairs_counts[['#query_name','paired_count']]
    
    colnames=('#query_name','length','single_count','whoknows')
    singleton_counts=pd.read_csv('{}/{}.singletons.idxstats'.format(align_dir,sample),sep="\t",names=colnames)
    singleton_counts=singleton_counts[['#query_name','single_count']]
    
    counts=pd.merge(proper_pairs_counts,singleton_counts,how='inner',on='#query_name')
    counts[sample]=counts['paired_count']+counts['single_count']
    counts=counts[['#query_name',sample]]
    
    return counts

Eggnog_mappers annotations are first read into a Pandas dataframe (`mappings`). `get_counts()` is then called for each sample in turn, with the returned values being added as an additional column of the `all_counts` dataframe. These two dataframes are then merged with an inner join on the '#query_name' column, and the resulting dataframe subsetted to retain the per-sample read-counts for each cDNA, and the associated GO terms.

In [190]:
samples=('2000', '2001', '2002', '2006', '2007', '2009', '2011', '2012', '2013', '2023', '2024', '2025')

cdna_dir='protein_clusters/alignments/'
eggnog_dir='eggnog_annotations/'

mappings=pd.read_csv('{}/eggnog.emapper.annotations'.format(eggnog_dir), sep="\t", header=1, skiprows=2)

all_counts=mappings['#query_name']
for sample in samples:
    counts=get_counts(cdna_dir,sample)
    all_counts=pd.merge(all_counts,counts,how='inner',on='#query_name')
    
merged_counts=pd.merge(all_counts,mappings,how='inner',on='#query_name')
merged_counts=merged_counts[['#query_name','GOs','2000','2001','2002','2006','2007','2009','2011','2012','2013','2023','2024','2025']]
merged_counts=merged_counts.dropna()
merged_counts.to_csv('{}/cdna_count_summary.txt'.format('protein_clusters'),sep='\t',index=True)

The `count_mispaired_cdna.py` script can now be used to count read-pairs mapped to different references, where the reference have >3 GO terms in common.

In [ ]:
!qsub -t 1-12 bin/count_mispaired_cdna.py --bam_dir protein_clusters/alignments/ --out_dir protein_clusters/invalid_pairs --eggnog eggnog_annotations/eggnog.emapper.annotations --common_terms 3

In [198]:
invalid_pairs_dir='protein_clusters/invalid_pairs/'

invalid_pairs=pd.DataFrame()
for sample in samples:
    if os.path.isfile('{}/{}.invalid_pairs.read_count_summary.txt'.format(invalid_pairs_dir,sample)):
        sample_df=pd.read_csv('{}/{}.invalid_pairs.read_count_summary.txt'.format(invalid_pairs_dir, sample), sep="\t",index_col=0)
    else:
        sample_df=pd.DataFrame(columns=[sample])
    invalid_pairs=pd.merge(invalid_pairs,sample_df,how='outer',left_index=True, right_index=True)
    
invalid_pairs.fillna('0',inplace=True)
invalid_pairs.to_csv('{}/all_valid_pairs.read_count_summary'.format(invalid_pairs_dir), sep="\t", index=True)

Now we can merge the counts of proper/singletons and improperly paired reads. Since we have counted read-pairs, improperly paired references may have 0.5, 1.5 etc. as counts. 

In [197]:
dir='protein_clusters/'

merged_counts=pd.read_csv('{}/cdna_count_summary.txt'.format(dir),sep='\t',header=0,index_col='#query_name',
                      dtype={'2000':float,'2001':float,'2002':float,'2006':float,'2007':float,'2009':float,
                             '2011':float,'2012':float,'2013':float,'2023':float,'2024':float,'2025':float})

invalid_pairs=invalid_pairs.astype({'2000':float,'2001':float,'2002':float,'2006':float,'2007':float,'2009':float,
                                    '2011':float,'2012':float, '2013':float,'2023':float,'2024':float,'2025':float})

combined_counts=merged_counts.merge(invalid_pairs,how='outer',left_index=True,right_index=True)
combined_counts.fillna(value=0,inplace=True)

for sample in samples:
    col1='{}_x'.format(sample)
    col2='{}_y'.format(sample)
    combined_counts[sample]=combined_counts[col1]+combined_counts[col2]
    combined_counts.drop([col1,col2],inplace=True,axis=1)
    
combined_counts.to_csv('{}/full_cdna_count_summary.txt'.format(dir),sep='\t',index=True)

We can now determine the number of counts for each term in each sample, with the 'count_go_terms.py' script...

In [ ]:
!qsub bin/count_go_terms.py --dir protein_clusters

## 7: GO SLIM mapping

Enrichment in individual GO terms is difficult to make sense of since there are so many terms changing due to the increase in bacterial content of the samples. 
GO Slims provide a more meaningful overview of what is going on, which requires mapping the identified GO terms to the metagenomics GO Slim. This is carried out using owltools. 


The GO basic ontology (data-version: releases/2020-03-23) and metagenomics GO slim were downloaded:

In [204]:
%%bash
mkdir GO_slim_mapping
wget -q -O GO_slim_mapping/go-basic.obo http://purl.obolibrary.org/obo/go/go-basic.obo 
wget -q -O GO_slim_mapping/goslim_metagenomics.obo http://current.geneontology.org/ontology/subsets/goslim_metagenomics.obo

Lists of ther terms in each obo format ontology were obtained using using `obo_to_tab.txt`, which produces tab-delimited text files as output, and a file containg a list of slim terms required by owltools (idfile) produced with awk.

In [216]:
%%bash
bin/obo_to_tab.py --infile GO_slim_mapping/go-basic.obo --outfile GO_slim_mapping/go_table.txt
bin/obo_to_tab.py --infile GO_slim_mapping/goslim_metagenomics.obo --outfile GO_slim_mapping/goslim_metagenomics_table.txt
cat GO_slim_mapping/goslim_metagenomics_table.txt|awk '{print $1}' > GO_slim_mapping/goslim_terms.txt

In [222]:
cluster_dir='protein_clusters/'
go_dir='GO_slim_mapping/'

terms_df=pd.read_csv('{}/go_count_table.txt'.format(cluster_dir),sep='\t',index_col=0)
terms=set(terms_df.index.values.tolist())
terms=list(terms)
gaf_file=open('{}/go_terms.gaf'.format(go_dir),'w')
gaf_file.write('!gaf-version: 2.0\n')
count=1
for term in terms:
    gaf_file.write('WB\t{}\t{}\toptional\t{}\tpubmed\tunknown\toptional\tunknown\toptional\toptional\tprotein\tunknown\t{}\tWB\toptional\toptional\n'.format(count,count,term,count))
    count+=1

The mapping can now be carried out using owltools:

In [ ]:
%%bash

owltools GO_slim_mapping/go-basic.obo --gaf GO_slim_mapping/go_terms.gaf --map2slim --idfile GO_slim_mapping/goslim_terms.txt --write-gaf GO_slim_mapping/go_slim_mapped_terms.gaf

A small number of terms were not mapped, so these can just be dropped from the results set. It should now be possible to merge the GAF files to produce a lookup table.

In [224]:
orig_terms=pd.read_csv('{}/go_terms.gaf'.format(go_dir),sep="\t",header=None,skiprows=1)
slim_terms=pd.read_csv('{}/go_slim_mapped_terms.gaf'.format(go_dir),sep="\t",header=None,skiprows=5)
orig_terms=orig_terms.iloc[:,[1,4]]
slim_terms=slim_terms.iloc[:,[1,4]]
slim_mapping=pd.merge(orig_terms,slim_terms,how='right',on=1)
slim_mapping=slim_mapping.iloc[:,[1,2]]
slim_mapping.columns=['GO','Slim']

#### GO Slim mappings  

Now redo the GO mapping using the GO slim terms...


In [ ]:
%%bash
qsub barley_filter/bin/go_slim.py --dir barley_filter/assembly_round2/protein_clusters